# SARIMAモデルによる乗客数の予測

In [ ]:
from passengers import Passengers, Line
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
file_name = 'data/t091307.xlsx'
line = '田園都市線'
skip_rows = list(range(0, 11)) + list(range(12, 15))
use_cols = list(range(1, 100))
denen = Line(file_name, line, skip_rows, use_cols)

ps = Passengers(line=denen)
display(ps.passengers.head())

# データの確認

## プロット

In [ ]:
df = ps.passengers['総数']
plt.figure(figsize=(16, 4))
plt.plot(df)
plt.show()

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(df.diff())
plt.show()

## 自己相関

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df, lags=24, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df, lags=24, ax=ax2)

In [ ]:
df_diff = df.diff()
df_diff = df_diff.dropna()

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df_diff, lags=24, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df_diff, lags=24, ax=ax2)

## 季節に分解

In [ ]:
def check_decompose(df, freq):
    res = sm.tsa.seasonal_decompose(df, freq=freq)
    original = df 
    trend = res.trend 
    seasonal = res.seasonal
    residual = res.resid
    
    plt.figure(figsize=(16, 8))
    plt.subplot(411)
    plt.plot(original)
    plt.ylabel('Original')
    
    plt.subplot(412)
    plt.plot(trend)
    plt.ylabel('Trend')
    
    plt.subplot(413)
    plt.plot(seasonal)
    plt.ylabel('Seasonality')
    
    plt.subplot(414)
    plt.plot(residual)
    plt.ylabel('Residuals')

    plt.tight_layout()
    plt.show()

In [ ]:
check_decompose(df, 12)

In [ ]:
plt.figure(figsize=(16, 2))
plt.plot(sm.tsa.seasonal_decompose(df, freq=12).seasonal)
plt.ylabel('Seasonality')
plt.xlim(['2015-01-01', '2016-01-01'])
plt.show()

In [ ]:
check_decompose(df, 6)

# SARIMAモデル

## 単位根過程の確認 

In [ ]:
adf = sm.tsa.stattools.adfuller(df, regression="ct")
print(adf)

adf_diff = sm.tsa.stattools.adfuller(df_diff, regression="ct")
print(adf_diff)

## ARMAモデルのパラメータ設定

In [ ]:
df_check = (df - sm.tsa.seasonal_decompose(df, freq=12).seasonal).diff().dropna()
plt.plot(df_check)
plt.show()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

res_selection = sm.tsa.arma_order_select_ic(df_check, ic='aic', trend='nc')
print(res_selection)

## SARIMAモデルに適用

In [ ]:
df_train = df[df.index < '2017-04-01']
df_test = df[df.index  >= '2017-04-01']

sarimax = sm.tsa.SARIMAX(df_train, 
                        order=(2, 1, 2),
                        seasonal_order=(1, 1, 1, 12),
                        enforce_stationarity = False,
                        enforce_invertibility = False
                        ).fit()

In [ ]:
sarimax_fore = sarimax.forecast(len(df_test) + 13) 

In [ ]:
sarimax_ci = sarimax.get_forecast(len(df_test) + 13).conf_int()

In [ ]:
plt.figure(figsize=(16, 4))

plt.plot(df, label="original")
plt.plot(sarimax_fore, label="sarimax-forecast")
plt.fill_between(sarimax_ci.index, sarimax_ci['lower 総数'], sarimax_ci['upper 総数'],facecolor='y',alpha=0.2)
plt.legend(loc='best')
plt.show()

In [ ]:
resid = sarimax.resid
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(resid, lags=24, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(resid, lags=24, ax=ax2)